<a href="https://colab.research.google.com/github/ashen-pabasara/fabric-defect-detection/blob/main/prediction_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [ ]:
# Install ultralytics
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import cv2
import glob
from ultralytics import YOLO
from tqdm import tqdm

# Mount Google Drive
drive.mount('/content/drive')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


Define Project Paths

In [ ]:
# Model Path
MODEL_PATH = "/content/drive/My Drive/EEY9536 Data Science Project/Dataset/YOLO_Results/yolov8n_colab_run_b64/weights/best.pt"

# Dataset ZIP Path
ZIP_PATH = "/content/drive/My Drive/EEY9536 Data Science Project/Dataset/fabric_dataset.zip"

UNZIP_DESTINATION = "/content/fabric_dataset"
SAMPLE_IMAGE_DIR = os.path.join(UNZIP_DESTINATION, "images", "val")
OUTPUT_SLIDESHOW_PATH = "/content/demo_slideshow.mp4"
LOCAL_RESULTS_DIR = "/content/prediction_results"

# Final Output Path
DRIVE_OUTPUT_DIR = "/content/drive/My Drive/EEY9536 Data Science Project/Dataset/Demo_Video/"
FINAL_VIDEO_NAME = "fabric_demo_with_boxes.mp4"

os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

print(f"Model Path: {MODEL_PATH}")
print(f"Dataset ZIP: {ZIP_PATH}")
print(f"Final video will be saved to: {DRIVE_OUTPUT_DIR}")

Model Path: /content/drive/My Drive/EEY9536 Data Science Project/Dataset/YOLO_Results/yolov8n_colab_run_b64/weights/best.pt
Dataset ZIP: /content/drive/My Drive/EEY9536 Data Science Project/Dataset/fabric_dataset.zip
Final video will be saved to: /content/drive/My Drive/EEY9536 Data Science Project/Dataset/Demo_Video/


Unzip Dataset

In [ ]:
print(f"Starting to unzip {ZIP_PATH} to /content/...")
!unzip -q -n "{ZIP_PATH}" -d "/content/"
print("Dataset unzipped successfully.")

Starting to unzip /content/drive/My Drive/EEY9536 Data Science Project/Dataset/fabric_dataset.zip to /content/...
Dataset unzipped successfully.


Create Slideshow Video

In [ ]:
import random

print("--- Part 1: Creating 'Slideshow' Video from DEFECTS ---")

labels_val_dir = os.path.join(UNZIP_DESTINATION, "labels", "val")
images_val_dir = os.path.join(UNZIP_DESTINATION, "images", "val")
NUM_IMAGES = 20

# Find all defective images
defective_image_files = []
all_label_files = os.listdir(labels_val_dir)

print(f"Scanning {len(all_label_files)} labels in {labels_val_dir}...")

for label_file in all_label_files:
    label_path = os.path.join(labels_val_dir, label_file)

    # Check if the label file is NOT empty
    if os.path.getsize(label_path) > 0:
        # Find its matching image
        image_name = label_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_val_dir, image_name)

        if os.path.exists(image_path):
            defective_image_files.append(image_path)

print(f"Found {len(defective_image_files)} defective images in the validation set.")

# Select 20 random samples
if len(defective_image_files) < NUM_IMAGES:
    print(f"Warning: Found fewer than 20 defect images. Using all {len(defective_image_files)}.")
    sample_images = defective_image_files
else:
    sample_images = random.sample(defective_image_files, NUM_IMAGES)

# Create the video
frame = cv2.imread(sample_images[0])
height, width, layers = frame.shape
size = (width, height)

# Define the video writer (1.0 FPS)
out = cv2.VideoWriter(OUTPUT_SLIDESHOW_PATH,
                      cv2.VideoWriter_fourcc(*'MP4V'),
                      1.0,
                      size)

print(f"Writing {len(sample_images)} defective images to {OUTPUT_SLIDESHOW_PATH}...")

for image_path in tqdm(sample_images, desc="Creating defect video"):
    img = cv2.imread(image_path)
    out.write(img)

out.release()
print("Defect slideshow video created successfully!")

--- Part 1: Creating 'Slideshow' Video from DEFECTS ---
Scanning 3948 labels in /content/fabric_dataset/labels/val...
Found 974 defective images in the validation set.
Writing 20 defective images to /content/demo_slideshow.mp4...


Creating defect video: 100%|██████████| 20/20 [00:00<00:00, 183.69it/s]

Defect slideshow video created successfully!


Run Model & Predict

In [ ]:
print("\n--- Part 2: Running Inference ---")

# Load trained model
print(f"Loading model from {MODEL_PATH}...")
model = YOLO(MODEL_PATH)
print("Model loaded successfully.")

# Run prediction
print(f"Running prediction on {OUTPUT_SLIDESHOW_PATH}...")
results = model.predict(
    source=OUTPUT_SLIDESHOW_PATH,
    save=True,
    conf=0.25, # Confidence threshold
    project=LOCAL_RESULTS_DIR,
    name="demo_run"
)
print("Prediction complete!")

actual_save_dir = results[0].save_dir

predicted_video_local_path = os.path.join(actual_save_dir, "demo_slideshow.avi")

final_drive_path = os.path.join(DRIVE_OUTPUT_DIR, FINAL_VIDEO_NAME)

# 4. Copy the video to Google Drive
print(f"Located predicted video at: {predicted_video_local_path}")
print(f"Copying to: {final_drive_path} ...")

!cp "{predicted_video_local_path}" "{final_drive_path}"

print("\n--- All Done! ---")
print(f"Successfully saved your final video to Google Drive at:")
print(f"{final_drive_path}")


--- Part 2: Running Inference ---
Loading model from /content/drive/My Drive/EEY9536 Data Science Project/Dataset/YOLO_Results/yolov8n_colab_run_b64/weights/best.pt...
Model loaded successfully.
Running prediction on /content/demo_slideshow.mp4...

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/20) /content/demo_slideshow.mp4: 512x512 1 Defect, 8.0ms
video 1/1 (frame 2/20) /content/demo_slideshow.mp4: 512x512 1 Defect, 6.9ms
video 1/1 (frame 3/20) /content/demo_slideshow.mp4: 512